In [20]:
import os
import json
import logging
logging.basicConfig(level=logging.INFO)
_logger = logging.getLogger(__name__) 
import requests

from jsonschema import validate, ValidationError
from ibm_watson import AssistantV2, ApiException
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from flask import jsonify
from flask_api import status


In [37]:
request_data = {
            "assistant_api_key": "EvG4FigjVLQbUgcjtgegK4XHfhv-Nn7r3iMtZBIo1xU8",
            "assistant_url": "https://api.us-south.assistant.watson.cloud.ibm.com/instances/2970b5ba-96c8-4342-b392-83f1e43efa31",
            "assistant_version": "2020-08-31",
            "assistant_id": "89eb5dd2-fbaf-4a28-a8f2-b8efe1ca69fb"
        }
    
def watson_create_session():


    
    iam_apikey = request_data.get("assistant_api_key")
    assistant_url = request_data.get("assistant_url")
    assistant_version = request_data.get('assistant_version')

    assistant = watson_instance(iam_apikey, assistant_url, assistant_version)

    try:
        watson_session = assistant.create_session(
            assistant_id=request_data.get("assistant_id")
        ).get_result()
        watson_session_id = watson_session["session_id"]
    except KeyError:
        _logger.error("The session wasn't created")
        return jsonify({"error": "Error creating the session"}), status.HTTP_503_SERVICE_UNAVAILABLE

    return watson_session_id


def watson_response(session_id, message):
    
    iam_apikey = request_data.get("assistant_api_key")
    assistant_url = request_data.get("assistant_url")
    assistant_version = request_data.get('assistant_version')

    assistant = watson_instance(iam_apikey, assistant_url, assistant_version)
    context = request_data.get('context') if 'context' in request_data else {}
    watson_session_id = session_id#request_data.get('session_id')

    try:
        watson_response = assistant.message(
            assistant_id=request_data.get('assistant_id'),
            session_id=watson_session_id,
            input={
                'message_type': 'text',
                'text': message,
                'options': {
                    'return_context': True
                }
            },
            context=context
        ).get_result()
    except ValueError as ex:
        _logger.error("Value error: %s", ex)
        return jsonify({'error': "Value error"}), status.HTTP_500_INTERNAL_SERVER_ERROR
    except ApiException:
        try:
            watson_session = assistant.create_session(
                assistant_id=request_data.get("assistant_id")
            ).get_result()
            watson_session_id = watson_session["session_id"]

            watson_response = assistant.message(
                assistant_id=request_data.get('assistant_id'),
                session_id=watson_session_id,
                input={
                    'message_type': 'text',
                    'text': request_data.get('input_message'),
                    'options': {
                        'return_context': True
                    }
                },
                context=context
            ).get_result()
        except KeyError:
            _logger.error("The session wasn't created")
            return jsonify({"error": "Error creating the session"}), status.HTTP_503_SERVICE_UNAVAILABLE

    try:
        del watson_response["context"]["global"]["session_id"]
    except:
        pass

    response = {
        "response": watson_response,
        "session_id": watson_session_id
    }

    return response

def watson_instance(iam_apikey: str, url: str, version: str = "2019-02-28") -> AssistantV2:
    try:
        authenticator = IAMAuthenticator(iam_apikey)
        assistant = AssistantV2(
            authenticator=authenticator,
            version=version
        )
        assistant.set_service_url(url)
    except ApiException as error:
        _logger.error("%s - %s", error.code, error.message)
        return jsonify({'error': str(error.message)}), error.code

    return assistant


In [38]:

session = watson_create_session()

In [39]:
session

'ac1d4529-cca7-422a-a54d-27dcdb0d37fb'

In [40]:
watson_response(session, 'hi')

{'response': {'output': {'intents': [{'intent': 'Salutations',
     'confidence': 1}],
   'entities': [],
   'generic': [{'response_type': 'text', 'text': 'Hi, how can I help?'}]},
  'context': {'global': {'system': {'turn_count': 1}},
   'skills': {'main skill': {'system': {}}}}},
 'session_id': 'ac1d4529-cca7-422a-a54d-27dcdb0d37fb'}

In [ ]:
#response["response"]["output"]["intents"][0]["intent"]